# Chapter 10 Analyzing the Meaning of Sentences

## 10.1 Natural Language Understanding

### Querying a Database

In [1]:
import nltk

In [2]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [5]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china"


In [6]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

## 10.2 Propositional Logic

In [7]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


In [9]:
read_expr = nltk.sem.Expression.fromstring

In [10]:
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [11]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [12]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [13]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

In [15]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [16]:
val['P']

True

In [17]:
dom = set([])

In [18]:
g = nltk.Assignment(dom)

In [19]:
m = nltk.Model(dom, val)

In [20]:
print(m.evaluate('(P & Q)', g))

True


In [21]:
print(m.evaluate('-(P & Q)', g))

False


In [22]:
print(m.evaluate('(P & R)', g))

False


In [23]:
print(m.evaluate('(P | R)', g))

True


## 10.3 First-Order Logic

### Syntax

In [25]:
read_expr = nltk.sem.Expression.fromstring

In [26]:
expr = read_expr('walk(angus)', type_check=True)

In [27]:
expr.argument

<ConstantExpression angus>

In [28]:
expr.argument.type

e

In [29]:
expr.function

<ConstantExpression walk>

In [30]:
expr.function.type

<e,?>

In [31]:
sig = {'walk': '<e, t>'}

In [32]:
expr = read_expr('walk(angus)', signature=sig)

In [33]:
expr.function.type

e

In [34]:
read_expr('dog(cyril)').free()

set()

In [35]:
read_expr('dog(x)').free()

{Variable('x')}

In [36]:
read_expr('own(angus, cyril)').free()

set()

In [37]:
read_expr('exists x.dog(x)').free()

set()

In [38]:
read_expr('((some x. walk(x)) -> sing(x))').free()

{Variable('x')}

In [39]:
read_expr('exists x.own(y, x)').free()

{Variable('y')}

### Summarizing the Language of First Order Logic

In [42]:
dom = {'b', 'o', 'c'}
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('c', 'b'), ('o', 'c')},
 'walk': {('c',), ('o',)}}


In [43]:
('o', 'c') in val['see']

True

In [44]:
('b',) in val['boy']

True

### Individual Variables and Assignments

In [45]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])

In [46]:
g

{'x': 'o', 'y': 'c'}

In [47]:
print(g)

g[c/y][o/x]


In [48]:
m = nltk.Model(dom, val)

In [49]:
m.evaluate('see(olive, y)', g)

True

In [50]:
g['y']

'c'

In [51]:
m.evaluate('see(y, x)', g)

False

In [52]:
g.purge()
g

{}

In [53]:
m.evaluate('see(olive, y)', g)

'Undefined'

In [54]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

True

### Quantification

In [55]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

True

In [56]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

True

In [57]:
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1, 'x', g)

{'b', 'o'}

In [58]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

{'b', 'c', 'o'}

In [59]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}

In [60]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

### Quantifier Scope Ambiguity

In [61]:
v2 = """
bruce => b
elspeth => e
julia => j
matthew => m
person => {b, e, j, m}
admire => {(j, b), (b, b), (m, e), (e, m)}
"""
val2 = nltk.Valuation.fromstring(v2)

In [62]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

In [63]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)

set()

In [64]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

## 10.4 The Semantics of English Sentences

###  The λ-Calculus

In [67]:
read_expr = nltk.sem.Expression.fromstring

In [68]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')

In [69]:
expr

<LambdaExpression \x.(walk(x) & chew_gum(x))>

In [71]:
expr.free()

set()

In [72]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

\x.(walk(x) & chew_gum(y))


In [73]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')

In [74]:
print(expr)

\x.(walk(x) & chew_gum(x))(gerald)


In [75]:
print(expr.simplify())

(walk(gerald) & chew_gum(gerald))


In [76]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())

\y.(dog(cyril) & own(y,cyril))


In [77]:
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

(dog(cyril) & own(angus,cyril))


In [78]:
expr1 = read_expr('exists x.P(x)')

In [80]:
print(expr1)

exists x.P(x)


In [81]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))

In [82]:
print(expr2)

exists z.P(z)


In [83]:
expr1 == expr2

True

In [84]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')

In [85]:
print(expr3)

(\P.exists x.P(x))(\y.see(y,x))


In [86]:
print(expr3.simplify())

exists z1.see(z1,x)


### Quantified NPs

In [87]:
read_expr = nltk.sem.Expression.fromstring
tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)

In [88]:
print(vp)

(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))


In [89]:
print(vp.simplify())

\x.exists z2.(dog(z2) & chase(x,z2))


In [90]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
sentence = 'Angus gives a bone to every dog'
tokens = sentence.split()
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

all z4.(dog(z4) -> exists z3.(bone(z3) & give(angus,z3,z4)))


In [92]:
>>> sents = ['Irene walks', 'Cyril bites an ankle']
>>> grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
>>> for results in nltk.interpret_sents(sents, grammar_file):
...     for (synrep, semrep) in results:
...         print(synrep)

(S[SEM=<walk(irene)>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(irene)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(irene)>] Irene))
  (VP[NUM='sg', SEM=<\x.walk(x)>]
    (IV[NUM='sg', SEM=<\x.walk(x)>, TNS='pres'] walks)))
(S[SEM=<exists z6.(ankle(z6) & bite(cyril,z6))>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(cyril)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(cyril)>] Cyril))
  (VP[NUM='sg', SEM=<\x.exists z6.(ankle(z6) & bite(x,z6))>]
    (TV[NUM='sg', SEM=<\X x.X(\y.bite(x,y))>, TNS='pres'] bites)
    (NP[NUM='sg', SEM=<\Q.exists x.(ankle(x) & Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] an)
      (Nom[NUM='sg', SEM=<\x.ankle(x)>]
        (N[NUM='sg', SEM=<\x.ankle(x)>] ankle)))))


In [93]:
>>> v = """
... bertie => b
... olive => o
... cyril => c
... boy => {b}
... girl => {o}
... dog => {c}
... walk => {o, c}
... see => {(b, o), (c, b), (o, c)}
... """
>>> val = nltk.Valuation.fromstring(v)
>>> g = nltk.Assignment(val.domain)
>>> m = nltk.Model(val.domain, val)
>>> sent = 'Cyril sees every boy'
>>> grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
>>> results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
>>> for (syntree, semrep, value) in results:
...     print(semrep)
...     print(value)

all z7.(boy(z7) -> see(cyril,z7))
True


### Quantifier Ambiguity Revisited

In [94]:
>>> from nltk.sem import cooper_storage as cs
>>> sentence = 'every girl chases a dog'
>>> trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')
>>> semrep = trees[0].label()['SEM']
>>> cs_semrep = cs.CooperStore(semrep)

In [95]:
print(cs_semrep.core)

chase(z2,z3)


In [96]:
>>> for bo in cs_semrep.store:
...     print(bo)

bo(\P.all x.(girl(x) -> P(x)),z2)
bo(\P.exists x.(dog(x) & P(x)),z3)


In [97]:
cs_semrep.s_retrieve(trace=True)

Permutation 1
   (\P.all x.(girl(x) -> P(x)))(\z2.chase(z2,z3))
   (\P.exists x.(dog(x) & P(x)))(\z3.all x.(girl(x) -> chase(x,z3)))
Permutation 2
   (\P.exists x.(dog(x) & P(x)))(\z3.chase(z2,z3))
   (\P.all x.(girl(x) -> P(x)))(\z2.exists x.(dog(x) & chase(z2,x)))


In [98]:
>>> for reading in cs_semrep.readings:
...     print(reading)

exists x.(dog(x) & all z10.(girl(z10) -> chase(z10,x)))
all x.(girl(x) -> exists z11.(dog(z11) & chase(x,z11)))


## 10.5 Discourse Semantics

### Discourse Representation Theory

In [101]:
>>> read_dexpr = nltk.sem.DrtExpression.fromstring
>>> drs1 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
>>> print(drs1)

([x,y],[angus(x), dog(y), own(x,y)])


In [102]:
print(drs1.fol())

exists x y.(angus(x) & dog(y) & own(x,y))


In [103]:
drs2 = read_dexpr('([x], [walk(x)]) + ([y], [run(y)])')

In [104]:
print(drs2)

(([x],[walk(x)]) + ([y],[run(y)]))


In [105]:
print(drs2.simplify())

([x,y],[walk(x), run(y)])


In [106]:
>>> drs3 = read_dexpr('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')
>>> print(drs3.fol())

all x.(dog(x) -> exists y.(ankle(y) & bite(x,y)))


In [107]:
 	
>>> drs4 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
>>> drs5 = read_dexpr('([u, z], [PRO(u), irene(z), bite(u, z)])')
>>> drs6 = drs4 + drs5

In [108]:
print(drs6.simplify())

([u,x,y,z],[angus(x), dog(y), own(x,y), PRO(u), irene(z), bite(u,z)])


In [109]:
print(drs6.simplify().resolve_anaphora())

([u,x,y,z],[angus(x), dog(y), own(x,y), (u = [x,y,z]), irene(z), bite(u,z)])


In [110]:
>>> from nltk import load_parser
>>> parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())
>>> trees = list(parser.parse('Angus owns a dog'.split()))
>>> print(trees[0].label()['SEM'].simplify())

([x,z14],[Angus(x), dog(z14), own(x,z14)])
